In [1]:
import numpy as np
import pycuda.autoinit
import pycuda.gpuarray as gpuarray
import skcuda.fft as cu_fft

/home/elvis/.local/lib/python3.10/site-packages/skcuda/cublas.py:284: UserWarning: creating CUBLAS context to get version number
  warnings.warn('creating CUBLAS context to get version number')


In [2]:
def fft2_gpu(x, fftshift=False):
    
    ''' This function produce an output that is 
    compatible with numpy.fft.fft2
    The input x is a 2D numpy array'''

    # Convert the input array to single precision float
    if x.dtype != 'float32':
        x = x.astype('float32')

    # Get the shape of the initial numpy array
    n1, n2 = x.shape
    
    # From numpy array to GPUarray
    xgpu = gpuarray.to_gpu(x)
    
    # Initialise output GPUarray 
    # For real to complex transformations, the fft function computes 
    # N/2+1 non-redundant coefficients of a length-N input signal.
    y = gpuarray.empty((n1,n2//2 + 1), np.complex64)
    
    # Forward FFT
    plan_forward = cu_fft.Plan((n1, n2), np.float32, np.complex64)
    cu_fft.fft(xgpu, y, plan_forward)
    
    left = y.get()

    # To make the output array compatible with the numpy output
    # we need to stack horizontally the y.get() array and its flipped version
    # We must take care of handling even or odd sized array to get the correct 
    # size of the final array   
    if n2//2 == n2/2:
        right = np.roll(np.fliplr(np.flipud(y.get()))[:,1:-1],1,axis=0)
    else:
        right = np.roll(np.fliplr(np.flipud(y.get()))[:,:-1],1,axis=0) 
    
    # Get a numpy array back compatible with np.fft
    if fftshift is False:
        yout = np.hstack((left,right))
    else:
        yout = np.fft.fftshift(np.hstack((left,right)))

    return yout.astype('complex128')  

In [3]:
def ifft2_gpu(y, fftshift=False):

    ''' This function produce an output that is 
    compatible with numpy.fft.ifft2
    The input y is a 2D complex numpy array'''
 
    # Get the shape of the initial numpy array
    n1, n2 = y.shape
    
    # From numpy array to GPUarray. Take only the first n2/2+1 non redundant FFT coefficients
    if fftshift is False:
        y2 = np.asarray(y[:,0:n2//2 + 1], np.complex64)
    else:
        y2 = np.asarray(np.fft.ifftshift(y)[:,:n2//2+1], np.complex64)
    ygpu = gpuarray.to_gpu(y2) 
     
    # Initialise empty output GPUarray 
    x = gpuarray.empty((n1,n2), np.float32)
    
    # Inverse FFT
    plan_backward = cu_fft.Plan((n1, n2), np.complex64, np.float32)
    cu_fft.ifft(ygpu, x, plan_backward)
    
    # Must divide by the total number of pixels in the image to get the normalisation right
    xout = x.get()/n1/n2
    
    return xout

In [4]:
# Set up input array
N = 1024
x = np.random.rand(N,N).astype(np.float32)

In [5]:
np.allclose(fft2_gpu(x),np.fft.fft2(np.fft.fftshift(x)))

False

In [6]:
%timeit -n 100 fft2_gpu(x)

7.09 ms ± 90.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%timeit -n 100 np.fft.fft2(x)

22.1 ms ± 488 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
def fft3_gpu(x, fftshift=False):
    
    ''' This function produces an output that is 
    compatible with numpy.fft.fftn
    The input x is a 3D numpy array'''

    # Convert the input array to single precision float
    if x.dtype != 'float32':
        x = x.astype('float32')

    # Get the shape of the initial numpy array
    n1, n2, n3 = x.shape
    
    # From numpy array to GPUarray
    xgpu = gpuarray.to_gpu(x)
    
    # Initialise output GPUarray
    # For real to complex transformations, the fft function computes 
    # (N/2+1) non-redundant coefficients of a length-N input signal.
    y = gpuarray.empty((n1, n2, n3//2 + 1), np.complex64)
    
    # Forward FFT
    plan_forward = cu_fft.Plan((n1, n2, n3), np.float32, np.complex64)
    cu_fft.fft(xgpu, y, plan_forward)
    
    left = y.get()

    # To make the output array compatible with the numpy output
    # we need to stack horizontally the y.get() array and its flipped version
    # We must take care of handling even or odd sized array to get the correct 
    # size of the final array   
    if n3//2 == n3/2:
        right = np.roll(np.fliplr(np.flipud(left))[:,:,1:-1],1,axis=1)
    else:
        right = np.roll(np.fliplr(np.flipud(left))[:,:,:-1],1,axis=1) 
    
    # Get a numpy array back compatible with np.fft
    if fftshift is False:
        yout = np.concatenate((left,right), axis=2)
    else:
        yout = np.fft.fftshift(np.concatenate((left,right), axis=2))

    return yout.astype('complex128')  

In [9]:
def ifft3_gpu(y, fftshift=False):

    ''' This function produces an output that is 
    compatible with numpy.fft.ifftn
    The input y is a 3D complex numpy array'''
 
    # Get the shape of the initial numpy array
    n1, n2, n3 = y.shape
    
    # From numpy array to GPUarray. Take only the first n3/2+1 non redundant FFT coefficients
    if fftshift is False:
        y2 = np.asarray(y[:,:,0:n3//2 + 1], np.complex64)
    else:
        y2 = np.asarray(np.fft.ifftshift(y)[:,:n3//2+1], np.complex64)
    ygpu = gpuarray.to_gpu(y2) 
     
    # Initialise empty output GPUarray 
    x = gpuarray.empty((n1,n2,n3), np.float32)
    
    # Inverse FFT
    plan_backward = cu_fft.Plan((n1, n2, n3), np.complex64, np.float32)
    cu_fft.ifft(ygpu, x, plan_backward)
    
    # Must divide by the total number of pixels in the image to get the normalisation right
    xout = x.get()/n1/n2/n3
    
    return xout


In [16]:
import numpy as np
import skcuda.fft as cu_fft
import pycuda.autoinit

# Set up input array
N = 256
x = np.random.rand(N,N,N).astype(np.float32)

In [17]:
y = fft3_gpu(x)

In [18]:
xout = ifft3_gpu(y)

In [19]:
np.isclose(xout,x)

array([[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  Tr

In [20]:
%timeit -n 100 fft3_gpu(x)

110 ms ± 2.83 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
%timeit -n 100 np.fft.fftn(x)

756 ms ± 4.44 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
import torch

In [26]:
def torch_fft3(data):
    return torch.fft(data, signal_ndim=3)

In [28]:
torch.fft.fftn(x)

TypeError: fft_fftn(): argument 'input' (position 1) must be Tensor, not numpy.ndarray